<a href="https://colab.research.google.com/github/hafezgh/MSc-Thesis/blob/main/decentralized_q_zerosum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Python code implementing Decentralized Q-learning Dynamics proposed in:

Sayin, Muhammed, et al. "Decentralized Q-learning in zero-sum Markov games." Advances in Neural Information Processing Systems 34 (2021).

# Experiment Setup

In [12]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(0)
n_actions = 3
n_states = 5
n_players = 2
init_state = np.random.randint(0,n_states)
current_state = init_state
next_state = None
rho = 0.7
rho_alpha = 0.9
rho_beta = 1.0
gamma = 0.6
D = 1/(1-gamma)
n_trials = 5
n_iters = int(1*1e7+1)
eps = 0.0002
tau = 2.0*np.ones((n_states))



q1 = np.zeros((n_states,n_actions))
q2 = np.zeros((n_states,n_actions))

v1 = np.zeros((n_states))
v2 = np.zeros((n_states))
vsum = np.zeros((n_states))


v1_mean = np.zeros((n_iters, n_states))
v2_mean = np.zeros((n_iters, n_states))
v_sum_mean = np.zeros((n_iters, n_states))

count_states = np.zeros((n_states), dtype=int)


S = np.array(list(range(n_states)), dtype=int)
A = np.array(list(range(n_actions)), dtype=int)


P = np.zeros((n_states,n_actions,n_actions, n_states))
R1 = np.zeros((n_states,n_actions,n_actions))
R2 = np.zeros((n_states,n_actions,n_actions))

a1 = np.random.choice(A)
a2 = np.random.choice(A)

r1 = 0.
r2 = 0.

pi_bar1 = np.ones((n_actions))
pi_bar1/=np.sum(pi_bar1)
pi_bar2 = np.ones((n_actions))
pi_bar2/=np.sum(pi_bar2)



In [13]:
# Define states, transition matrix, rewards
for i in range(len(S)):
    for j in range(len(A)):
        for k in range(len(A)):
            P[i, j, k, :] = np.random.uniform(0,1,(n_states)) + 0.01
            P[i, j, k, :] /= np.sum(P[i, j, k, :])

for i in range(len(S)):
    R1[i,:,:] = np.random.uniform(-1,1,(n_actions,n_actions))*np.exp(S[i]**2)
    R2[i,:,:] = -R1[i,:,:]

    max_r = np.max(abs(R1[i,:,:]))
    R1[i,:,:] /= max_r
    R2[i,:,:] /= max_r



# Utility Functions

In [14]:
def tau_decay_to_zero(count_state, rho_alpha,rho,D):
    tau_bar = 0.068
    return tau_bar/(1+(tau_bar*rho_alpha*rho*np.log(count_state))/(4*D))

def tau_decay_to_eps(count_state, eps):
    #tau_bar = 45000
    tau_bar = 4500
    return (1/count_state)*tau_bar+(1-1/count_state)*eps

def compute_alpha(count_state, rho_alpha):
    C = 1.5
    return C*1/pow(count_state,rho_alpha)

def compute_beta(count_state, rho_beta):
    C = 1.5
    return C*1/pow(count_state,rho_beta)

def softmax(arr):
    probs = np.exp(arr)/np.sum(np.exp(arr))
    return np.argmax(probs), np.max(probs), probs


# Run the experiment

In [11]:
counter = 0
for trial in range(n_trials):
    init_state = np.random.randint(0,n_states)
    current_state = init_state
    
    next_state = None
    tau = 2.0*np.ones((n_states))
    q1 = np.zeros((n_states,n_actions))
    q2 = np.zeros((n_states,n_actions))
    v1 = np.zeros((n_states))
    v2 = np.zeros((n_states))
    vsum = np.zeros((n_states))
    
    v1_hist = np.zeros((n_iters, n_states))
    v2_hist = np.zeros((n_iters, n_states))
    vsum_hist = np.zeros((n_iters, n_states))
    

    count_states = np.zeros((n_states), dtype=int)
    print('trial:', trial+1)
    
    for iter in range(n_iters):
        count_states[current_state] += 1
        
        if iter % int(1e5) == 0 and iter!=0:
            print("iteration:", iter)

        # take actions and update tau
        a1, a1_prob, pi_bar1 = softmax(q1[current_state,:]/tau[current_state])
        a2, a2_prob, pi_bar2 = softmax(q2[current_state,:]/tau[current_state])

        ## Decay to epsilone
        tau[current_state] = tau_decay_to_eps(count_states[current_state], eps)

        # Decay to zero
        #tau[current_state] = tau_decay_to_zero(count_states[current_state], rho_alpha,rho, D)

        # calculate rewards
        r1 = R1[current_state, a1, a2]
        r2 = R2[current_state, a1, a2]

        # next state
        next_state = np.random.choice(list(range(n_states)),p=P[current_state, a1, a2, :])

        # calculate learning rates (alpha and beta)
        alpha_bar1 = min(1, compute_alpha(count_states[current_state],rho_alpha)/a1_prob)
        alpha_bar2 = min(1, compute_alpha(count_states[current_state],rho_alpha)/a2_prob)
        beta = compute_beta(count_states[current_state], rho_beta)

        # update local q-function and value functions
        q1[current_state, a1] = q1[current_state, a1] + alpha_bar1*(r1+gamma*v1[next_state]-q1[current_state,a1])
        v1[current_state] = v1[current_state] + beta*(a1_prob*q1[current_state, a1]-v1[current_state])

        q2[current_state, a2] = q2[current_state, a2] + alpha_bar2*(r2+gamma*v2[next_state]-q2[current_state,a2])
        v2[current_state] = v2[current_state] + beta*(a2_prob*q2[current_state, a2]-v2[current_state])
        
        v1_hist[iter,:] = v1
        v2_hist[iter,:] = v2
        vsum = v1+v2
        vsum_hist[iter, :] = vsum
        current_state = copy.deepcopy(next_state)

    v1_mean = (v1_mean*counter + np.array(v1_hist))/(counter+1)
    v2_mean = (v2_mean*counter + np.array(v2_hist))/(counter+1)
    v_sum_mean = (v_sum_mean*counter + np.array(vsum_hist))/(counter+1)
    counter+=1

    

# Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.plot(list(range(n_iters)),v1_mean[:,0], color='red')
ax.plot(list(range(n_iters)),v1_mean[:,1], color='blue')
ax.plot(list(range(n_iters)),v1_mean[:,2], color='pink')
ax.plot(list(range(n_iters)),v1_mean[:,3], color='green')
ax.plot(list(range(n_iters)),v1_mean[:,4], color='orange')

ax.plot(list(range(n_iters)),v2_mean[:,0], color='red')
ax.plot(list(range(n_iters)),v2_mean[:,1], color='blue')
ax.plot(list(range(n_iters)),v2_mean[:,2], color='pink')
ax.plot(list(range(n_iters)),v2_mean[:,3], color='green')
ax.plot(list(range(n_iters)),v2_mean[:,4], color='orange')

ax.plot(list(range(n_iters)),v_sum_mean[:,0], color='black')
ax.plot(list(range(n_iters)),v_sum_mean[:,1], color='black')
ax.plot(list(range(n_iters)),v_sum_mean[:,2], color='black')
ax.plot(list(range(n_iters)),v_sum_mean[:,3], color='black')
ax.plot(list(range(n_iters)),v_sum_mean[:,4], color='black')

plt.show()